In [2]:
'''
Code provided by Philippine Space Agency on March 15 to 17 2023
plotting section edited by S Visaga
to use cartopy instead of basemap
'''

import netCDF4
import numpy as np
#from mpl_toolkits.basemap import Basemap

import matplotlib.pyplot as plt
import glob
import os
import time

import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [3]:
### Function to plot basemap and lon lat labels ###
def plot_background(ax):
    ax.add_feature(cf.LAKES.with_scale('10m'),facecolor='none', edgecolor='black',linewidth=0.8)
    ax.add_feature(cf.COASTLINE.with_scale('10m'),facecolor='none', edgecolor='black', linewidth=1) 
    return ax

def plot_ticks(ax):
    ax.set_yticks(np.arange(-5, 22, 5), crs=ccrs.PlateCarree())
    ax.set_xticks(np.arange(95, 130, 5), crs=ccrs.PlateCarree())
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    return ax

In [4]:
#List all netcd4 files in the directory using glob to filter ".nc" file extensions
l_files = glob.glob("/ships22/raqms/Satellite/GEMS/L3/NO2/*.nc")
print(l_files)

['/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230809_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230808_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230831_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230802_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230830_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230803_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230806_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230807_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230805_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230804_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230819_NO2_GRID-EA_daily_010.nc', '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_202

In [8]:
#Create a list for date, time, NO2 data, lat and lon that will be extracted from the nc files
l_dates = list()
l_times = list()
l_ColumnAmountNO2Trop = list()
l_lat = list()
l_lon = list()

In [9]:
# Loop for extraction of information from the netCDF4 files contained in l_files
for i in range(0, len(l_files)):
    print(i)
    # Get date and time information from the filename
    s_temp = str(l_files[i])
    sdate = s_temp.split('_')[3]  # date information
    stime = s_temp.split('_')[4]  # time information
    l_dates.append(sdate)
    l_times.append(stime)

    # Get variables in the netCDF4
    src = netCDF4.Dataset(l_files[i])
    data_fields = src.groups['Data Fields']
    print('Dataset', src)
    print('Data variables', data_fields)

    # Get NO2 data, latitude and longitude
    ColumnAmountNO2 = src.groups['Data Fields'].variables['ColumnAmountNO2']
    ColumnAmountNO2Trop = src.groups['Data Fields'].variables['ColumnAmountNO2Trop']
    lat = src.groups['Geolocation Fields'].variables['Latitude']
    lon = src.groups['Geolocation Fields'].variables['Longitude']

    # Use fallback fill value if not provided
    fv = ColumnAmountNO2Trop.getncattr('_FillValue') if '_FillValue' in ColumnAmountNO2Trop.ncattrs() else -9999
    print('Column Amount NO2 Tropospheric Metadata', ColumnAmountNO2Trop, 'Filling Value', fv)
    # Get NO2 data, latitude and longitude for total column NO2 instead of tropospheric
    fv_NO2 = ColumnAmountNO2.getncattr('_FillValue') if '_FillValue' in ColumnAmountNO2.ncattrs() else -9999
    print('Column Amount NO2 Metadata', ColumnAmountNO2, 'Filling Value', fv_NO2)

    # NO2 data scaling to 10^16 and fill values
    m_ColumnAmountNO2Trop = np.fliplr(np.flipud(np.array(ColumnAmountNO2Trop[:])))
    m_ColumnAmountNO2Trop[m_ColumnAmountNO2Trop == fv] = np.nan
    m_ColumnAmountNO2Trop = m_ColumnAmountNO2Trop / 1e16

    # Output data to list
    l_ColumnAmountNO2Trop.append(m_ColumnAmountNO2Trop)

    # Geolocation values
    m_lat = np.fliplr(np.flipud(np.array(lat[:])))
    m_lon = np.fliplr(np.flipud(np.array(lon[:])))
    m_lat[m_lat == -fv] = np.nan
    m_lon[m_lon == fv] = np.nan

    # Output data to list
    l_ColumnAmountNO2Trop.append(m_ColumnAmountNO2Trop)
    l_lat.append(m_lat)
    l_lon.append(m_lon)


0
Dataset <class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): image(540), spatial(750)
    variables(dimensions): 
    groups: Geolocation Fields, Data Fields
Data variables <class 'netCDF4.Group'>
group /Data Fields:
    dimensions(sizes): 
    variables(dimensions): float64 ColumnAmountNO2(image, spatial), float64 ColumnAmountNO2Trop(image, spatial)
    groups: 
Column Amount NO2 Tropospheric Metadata <class 'netCDF4.Variable'>
float64 ColumnAmountNO2Trop(image, spatial)
path = /Data Fields
unlimited dimensions: 
current shape = (540, 750)
filling on, default _FillValue of 9.969209968386869e+36 used Filling Value -9999
Column Amount NO2 Metadata <class 'netCDF4.Variable'>
float64 ColumnAmountNO2(image, spatial)
path = /Data Fields
unlimited dimensions: 
current shape = (540, 750)
filling on, default _FillValue of 9.969209968386869e+36 used Filling Value -9999
1
Dataset <class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file forma

In [10]:
print(l_lon[0].shape)


(540, 750)


In [11]:
print(src)
print(src.groups['Data Fields'].variables.keys())

<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): image(540), spatial(750)
    variables(dimensions): 
    groups: Geolocation Fields, Data Fields
dict_keys(['ColumnAmountNO2', 'ColumnAmountNO2Trop'])


In [12]:
m_ColumnAmountNO2Trop[3].shape

(750,)

In [13]:
# Subset areas
masked_lat = list()
masked_lon = list()

n = min(len(l_ColumnAmountNO2Trop), len(l_lat), len(l_lon))
for i in range(n):
    temp_lat = l_lat[i].copy()
    temp_lat[temp_lat < -5.0] = np.nan
    temp_lat[temp_lat > 22.0] = np.nan

    temp_lon = l_lon[i].copy()
    temp_lon[temp_lon < 95.0] = np.nan
    temp_lon[temp_lon > 128.0] = np.nan

    masked_lat.append(temp_lat.copy())
    masked_lon.append(temp_lon.copy())

In [14]:
#Apply subset to NO2 data
masked_ColumnAmountNO2Trop = list()

for i in range(n):
    #print(i)
    temp_ColumnAmountNO2Trop = l_ColumnAmountNO2Trop[i].copy()
    temp_ColumnAmountNO2Trop[np.isnan(masked_lat[i])] = np.nan
    temp_ColumnAmountNO2Trop[np.isnan(masked_lon[i])] = np.nan
    temp_ColumnAmountNO2Trop[temp_ColumnAmountNO2Trop < 0] = np.nan
    masked_ColumnAmountNO2Trop.append(temp_ColumnAmountNO2Trop.copy())

In [15]:
lat_min = np.nanmin(masked_lat[i])
lat_max = np.nanmax(masked_lat[i])
lon_min = np.nanmin(masked_lon[i])
lon_max = np.nanmax(masked_lon[i])

In [20]:
# --------- Helper Functions ---------
def plot_background(ax):
    ax.add_feature(cf.LAKES.with_scale('10m'), facecolor='none', edgecolor='black', linewidth=0.8)
    ax.add_feature(cf.COASTLINE.with_scale('10m'), facecolor='none', edgecolor='black', linewidth=1)
    ax.add_feature(cf.BORDERS.with_scale('10m'), linestyle=':')
    return ax

def plot_ticks(ax, lon_min, lon_max, lat_min, lat_max):
    ax.set_xticks(np.arange(np.floor(lon_min), np.ceil(lon_max) + 1, 10), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(np.floor(lat_min), np.ceil(lat_max) + 1, 10), crs=ccrs.PlateCarree())
    ax.xaxis.set_major_formatter(LongitudeFormatter(number_format='.0f', degree_symbol='°'))
    ax.yaxis.set_major_formatter(LatitudeFormatter(number_format='.0f', degree_symbol='°'))
    return ax

# --------- File Loop Setup ---------
files = sorted(glob.glob("/ships22/raqms/Satellite/GEMS/L3/NO2/*.nc"))
output_dir = "GEMS_Trop_NO2_Figures"
os.makedirs(output_dir, exist_ok=True)

for f in files:
    try:
        print("Processing:", os.path.basename(f))
        src = netCDF4.Dataset(f)

        # Extract data
        no2 = np.array(src.groups['Data Fields'].variables['ColumnAmountNO2Trop'][:])
        lat = np.array(src.groups['Geolocation Fields'].variables['Latitude'][:])
        lon = np.array(src.groups['Geolocation Fields'].variables['Longitude'][:])

        # Get and apply fill value
        fv = src.groups['Data Fields'].variables['ColumnAmountNO2Trop'].getncattr('_FillValue') if '_FillValue' in src.groups['Data Fields'].variables['ColumnAmountNO2Trop'].ncattrs() else -9999
        no2 = np.where(no2 == fv, np.nan, no2 / 1e16)  # convert to 1e16 molec/cm²

        # Get bounds
        lon_min, lon_max = lon.min(), lon.max()
        lat_min, lat_max = lat.min(), lat.max()

        # Extract date string from filename
        filename = os.path.basename(f)
        date_str = filename.split('_')[3]  # e.g., "20230809"
        title_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"  # → "2023-08-09"

        # Plot
        fig = plt.figure(figsize=(10, 6), dpi=150)
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
        plot_background(ax)
        plot_ticks(ax, lon_min, lon_max, lat_min, lat_max)

        mesh = ax.pcolormesh(lon, lat, no2, transform=ccrs.PlateCarree(),
                             cmap='gist_stern_r', vmin=0, vmax=3)

        plt.colorbar(mesh, orientation='horizontal',
                     label='NO₂ (10¹⁶ molecules/cm²)', fraction=0.046, pad=0.1)
        ax.set_title(f"GEMS Troposphere Column NO₂ – {title_date}")
        plt.savefig(f"{output_dir}/NO2_{date_str}.png", facecolor='white', bbox_inches='tight')
        plt.close()

    except Exception as e:
        print(f"Skipping {f} due to error: {e}")

Processing: GK2_GEMS_L3_20230802_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230803_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230804_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230805_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230806_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230807_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230808_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230809_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230810_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230811_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230812_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230813_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230814_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230815_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230816_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230817_NO2_GRID-EA_daily_010.nc
Processing: GK2_GEMS_L3_20230818_NO2_GRID-EA_daily_010.nc
Processing: GK